<a href="https://colab.research.google.com/github/cs-iuu/word-sense-2025-fall-ai/blob/main/notebooks/15.1.wsi_en_cht.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Step 2: Preprocessing: Extract Common Nouns

In [ ]:
!pip install spacy jieba pandas --break-system-packages
!python -m spacy download en_core_web_sm

In [5]:
"""
Step 2: Preprocessing
======================
Tokenizes, POS-tags, and lemmatizes both corpora.
Extracts common nouns only (no proper nouns, pronouns, or stopwords).
Applies a minimum frequency threshold to filter rare words.

Outputs:
  - data/english_nouns.csv   : lemma, verse_id, token, context (full verse)
  - data/chinese_nouns.csv   : lemma, verse_id, token, context
  - data/english_noun_freq.csv
  - data/chinese_noun_freq.csv

Usage:
  pip install spacy jieba pandas --break-system-packages
  python -m spacy download en_core_web_sm
  python 02_preprocessing.py

Design decisions (paper §3.2):
  - English: spaCy en_core_web_sm for tokenization, POS, lemmatization
  - Chinese: jieba for word segmentation + custom POS (jieba.posseg)
  - POS filters: English NOUN tag; Chinese POS prefix 'n' (common noun)
  - Proper noun exclusion: English PROPN tag excluded; Chinese 'nr','ns','nt','nz' excluded
  - Minimum frequency: MIN_FREQ = 30 (ensures sufficient WSI context)
  - Stopwords: NLTK English stopwords; custom Chinese stopword list
"""

import re
import pandas as pd
from pathlib import Path
from collections import Counter

# ─── Configuration ────────────────────────────────────────────────────────────

DATA_DIR = Path("/content") / "bible_data"
MIN_FREQ = 30          # Minimum occurrences per lemma for WSI
MAX_CONTEXT_LEN = 512  # Characters — prevents overlong inputs to transformers

# Chinese POS tags for common nouns (jieba.posseg notation)
ZH_NOUN_PREFIXES = {"n"}           # Common noun prefix
ZH_EXCLUDE_TAGS  = {"nr", "ns", "nt", "nz", "nw"}  # Proper nouns to exclude

# ── Theological proper noun exclusion lists ───────────────────────────────────
# These terms are proper nouns in English (God, Lord, Christ etc.) — excluded
# by spaCy's PROPN tag — but are tagged as common nouns n by jieba in Chinese
# due to the absence of capitalisation. They must be excluded explicitly from
# the Chinese data to ensure cross-lingual comparability.
#
# English side: spaCy correctly tags God/Lord/Christ as PROPN (excluded).
# Exception: "Spirit" (Holy Spirit) is sometimes tagged NOUN by spaCy, so it
# is added to EN_THEOLOGICAL_EXCLUDE as a lemma-level backstop.
#
# Borderline cases kept in both languages:
#   先知/prophet  — generic occupational noun, polysemous, common in both
#   天使/angel    — generic supernatural being, common noun in both
#   魔鬼/devil    — common noun in EN; jieba tags n in ZH

ZH_THEOLOGICAL_EXCLUDE = {
    # Core deity names / titles
    "神",     # God (most frequent — 1244 occurrences)
    "主",     # Lord
    "上帝",   # God (formal)
    "耶和華", # Yahweh / LORD
    "基督",   # Christ
    "耶穌",   # Jesus (also usually tagged nr, but belt-and-suspenders)
    "聖靈",   # Holy Spirit
    "聖神",   # Holy Spirit (alternate form in some CUV editions)
    "彌賽亞", # Messiah
    # Adversarial proper nouns
    "撒但",   # Satan
    "別西卜", # Beelzebub
}

EN_THEOLOGICAL_EXCLUDE = {
    # Lemma-level backstop for cases where spaCy tags as NOUN not PROPN
    "spirit",    # "Holy Spirit" — spaCy inconsistently tags as NOUN
    "ghost",     # "Holy Ghost" (KJV form; rare in NIV but present)
}

# Path to custom jieba dictionary for biblical proper names
JIEBA_DICT_PATH = Path("/content") / "bible_data" / "jieba_biblical_dict.txt"

# ── Post-segmentation POS correction ─────────────────────────────────────────
# jieba's POS tagger runs independently of the segmentation dictionary and
# can assign incorrect tags even for dictionary entries. These words are
# forced to tag n after segmentation regardless of what the POS tagger assigned.
#
# 地: jieba assigns uv (虛詞/copular particle) in classical subject-predicate
#     constructions like 地是空虛混沌 (Gen.1.2) because it parses 地 as a
#     topic marker rather than a subject noun. This is a known jieba limitation
#     with literary Chinese. Since English "earth" (freq=739) is always tagged
#     NOUN by spaCy, forcing 地 to n is required for cross-lingual comparability.
ZH_FORCE_NOUN_TAG = {
    "地",   # earth/ground/land — incorrectly tagged uv in copular constructions
}

# ─── English Preprocessing ────────────────────────────────────────────────────


def preprocess_english(verse_csv: Path) -> pd.DataFrame:
    """
    Process English verses with spaCy.
    Returns long-format DataFrame: one row per noun occurrence.
    """
    try:
        import spacy
    except ImportError:
        raise ImportError("Run: pip install spacy --break-system-packages && python -m spacy download en_core_web_sm")

    print("  [EN] Loading spaCy model…")
    nlp = spacy.load("en_core_web_sm", disable=["ner", "textcat"])

    df = pd.read_csv(verse_csv)
    print(f"  [EN] Processing {len(df):,} verses…")

    import time
    records   = []
    texts     = df["text"].tolist()
    verse_ids = df["verse_id"].tolist()
    total     = len(texts)
    t0        = time.time()

    for i, (doc, vid) in enumerate(zip(nlp.pipe(texts, batch_size=512), verse_ids), 1):
        context = doc.text[:MAX_CONTEXT_LEN]
        for token in doc:
            # Keep only common nouns; exclude proper nouns and pronouns
            if (
                token.pos_ == "NOUN"
                and not token.is_stop
                and not token.is_punct
                and len(token.lemma_) > 1
                and token.lemma_.isalpha()
                and token.lemma_.lower() not in EN_THEOLOGICAL_EXCLUDE
            ):
                records.append({
                    "verse_id": vid,
                    "token":    token.text,
                    "lemma":    token.lemma_.lower(),
                    "context":  context,
                })
        if i % 1000 == 0 or i == total:
            elapsed = time.time() - t0
            rate    = i / elapsed if elapsed > 0 else 0
            eta_min = (total - i) / rate / 60 if rate > 0 else 0
            print(f"    … {i:,}/{total:,} verses  "
                  f"({rate:.0f} v/s)  "
                  f"ETA {eta_min:.1f} min  "
                  f"nouns so far: {len(records):,}",
                  end="\r")

    elapsed_total = time.time() - t0
    print(f"\n  [EN] Done in {elapsed_total/60:.1f} min. "
          f"Extracted {len(records):,} noun occurrences.")
    return pd.DataFrame(records)


# ─── Chinese Preprocessing ────────────────────────────────────────────────────


def preprocess_chinese(verse_csv: Path) -> pd.DataFrame:
    """
    Process Chinese verses with jieba (word segmentation + POS tagging).
    Returns long-format DataFrame: one row per noun occurrence.

    Performance notes:
    - Uses itertuples() instead of iterrows() — ~10x faster for row iteration
    - Progress printed every 1000 verses so you can monitor speed
    - jieba is single-threaded by design; no further parallelism available
      without multiprocessing (not needed — full Bible processes in ~5 min)

    Note: jieba lemma == surface form for Chinese (no inflection).
    """
    try:
        import jieba
        import jieba.posseg as pseg
    except ImportError:
        raise ImportError("Run: pip install jieba --break-system-packages")

    # Load custom dictionary BEFORE any segmentation.
    # This ensures biblical proper names (亞伯拉罕, 以色列 etc.) are treated
    # as single tokens tagged nr/ns rather than being split into fragments
    # that jieba mis-tags as common nouns n.
    if JIEBA_DICT_PATH.exists():
        jieba.load_userdict(str(JIEBA_DICT_PATH))
        print(f"  [ZH] Loaded biblical proper name dictionary: {JIEBA_DICT_PATH.name}")
    else:
        print(f"  [ZH] WARNING: custom dictionary not found at {JIEBA_DICT_PATH}")
        print(f"       Proper names may be mis-segmented. Check data/ folder.")

    # Suppress jieba's per-sentence initialisation messages
    jieba.setLogLevel("ERROR")

    # Load custom stopwords for Chinese
    zh_stopwords = _load_chinese_stopwords()

    df = pd.read_csv(verse_csv)
    total = len(df)
    print(f"  [ZH] Processing {total:,} verses with jieba…")
    import time
    t0 = time.time()

    records = []
    # itertuples() is ~10x faster than iterrows() — avoids per-row Series creation
    for i, row in enumerate(df.itertuples(index=False), 1):
        verse_id = row.verse_id
        text     = str(row.text)
        context  = text[:MAX_CONTEXT_LEN]

        for word, flag in pseg.cut(text):
            flag_str = str(flag)
            # ── POS correction for known jieba mistagging in classical CUV ──
            # jieba's POS tagger assigns uv (copular particle) to 地 in
            # classical subject-predicate constructions (e.g. 地是空虛混沌).
            # We force n to match English "earth" which spaCy always tags NOUN.
            if word in ZH_FORCE_NOUN_TAG:
                flag_str = "n"
            # Accept common nouns only; skip proper nouns and theological terms
            if (
                flag_str[:1] in ZH_NOUN_PREFIXES
                and flag_str not in ZH_EXCLUDE_TAGS
                and word not in zh_stopwords
                and word not in ZH_THEOLOGICAL_EXCLUDE
                and len(word) >= 1
                and not word.isdigit()
            ):
                records.append({
                    "verse_id": verse_id,
                    "token":    word,
                    "lemma":    word,
                    "context":  context,
                })

        # Progress every 1000 verses — shows verses/sec so you know it's running
        if i % 1000 == 0 or i == total:
            elapsed  = time.time() - t0
            rate     = i / elapsed if elapsed > 0 else 0
            eta_secs = (total - i) / rate if rate > 0 else 0
            eta_min  = eta_secs / 60
            print(f"    … {i:,}/{total:,} verses  "
                  f"({rate:.0f} v/s)  "
                  f"ETA {eta_min:.1f} min  "
                  f"nouns so far: {len(records):,}",
                  end="\r")

    elapsed_total = time.time() - t0
    print(f"\n  [ZH] Done in {elapsed_total/60:.1f} min. "
          f"Extracted {len(records):,} noun occurrences.")
    return pd.DataFrame(records)


def _load_chinese_stopwords() -> set:
    """
    Chinese function word stoplist for CUV Traditional (CHT) text.

    Design decisions:
    ─────────────────────────────────────────────────────────────
    1. CHT variants included alongside CHS equivalents for all
       characters that differ between scripts (說/说, 會/会, etc.)

    2. 人 is NOT a stopword. It is a genuine common noun meaning
       "person / people / man / humanity" and is highly polysemous
       in biblical text. Jieba tags it as n (common noun) in most
       contexts, so it passes the POS filter correctly. Removing it
       would discard one of the most semantically rich words in the
       corpus.

    3. Pronouns (他/她/祂/你/我 etc.) are NOT listed here. They are
       tagged by jieba as r (pronoun), which is already excluded by
       the POS filter (we keep only n* tags). Listing them would be
       redundant. The various gendered and honorific variants
       (他/她/它/祂) all carry the r tag and are excluded uniformly.

    4. This list covers only high-frequency grammatical function
       words that jieba may occasionally mis-tag as nouns.
       It is intentionally conservative.
    ─────────────────────────────────────────────────────────────
    """
    return {
        # Structural particles (occasionally mis-tagged as n by jieba)
        # NOTE: 地 is intentionally NOT listed here.
        # In CUV literary style, 地 is overwhelmingly used as a noun
        # (earth/land/ground) matching English "earth" (freq=739).
        # The adverbial particle use of 地 is rare in classical biblical text.
        # Removing it would create an asymmetry with English where "earth"
        # is correctly retained as a high-frequency common noun.
        "的", "得",
        # Aspect markers — CHT: 著, CHS: 着
        "了", "著", "着",
        # Conjunctions / connectives
        "和", "與", "与", "及", "或", "但", "而", "且",
        # Adverbs sometimes mis-tagged
        "也", "都", "就", "才", "又", "還", "还", "已",
        "很", "更", "最", "太", "非常",
        # Negation
        "不", "沒有", "没有", "未", "無", "无",
        # Existential / copular
        "是", "有", "在",
        # Determiners / quantifiers
        "一", "這", "这", "那", "各", "每", "某", "其",
        # Directional / locative words with no sense variation
        "上", "下", "中", "內", "内", "外", "前", "後", "后",
        "裡", "里", "間", "间",
        # Common verbs jieba occasionally tags as nouns in CUV
        "說", "说", "看", "去", "來", "来", "到", "給", "给",
        "要", "會", "会",
    }


# ─── Frequency Filtering ──────────────────────────────────────────────────────


def apply_frequency_filter(df: pd.DataFrame, min_freq: int = MIN_FREQ) -> tuple:
    """
    Keep only lemmas appearing at least `min_freq` times.
    Returns (filtered_df, freq_df).
    """
    freq = df.groupby("lemma").size().reset_index(name="count")
    freq = freq.sort_values("count", ascending=False)
    valid_lemmas = set(freq[freq["count"] >= min_freq]["lemma"])
    filtered = df[df["lemma"].isin(valid_lemmas)].copy()
    return filtered, freq


# ─── Main ─────────────────────────────────────────────────────────────────────


# def main():
#     print("=" * 60)
#     print("Step 2: Preprocessing")
#     print("=" * 60)

#     # ── English ──────────────────────────────────────────────────
#     en_raw = preprocess_english(DATA_DIR / "english_verses.csv")
#     en_filtered, en_freq = apply_frequency_filter(en_raw)
#     en_filtered.to_csv(DATA_DIR / "english_nouns.csv", index=False, encoding="utf-8")
#     en_freq.to_csv(DATA_DIR / "english_noun_freq.csv", index=False, encoding="utf-8")
#     print(f"  [EN] {en_filtered['lemma'].nunique():,} lemmas ≥ {MIN_FREQ} occurrences retained.")

#     # ── Chinese ───────────────────────────────────────────────────
#     zh_raw = preprocess_chinese(DATA_DIR / "chinese_verses.csv")
#     zh_filtered, zh_freq = apply_frequency_filter(zh_raw)
#     zh_filtered.to_csv(DATA_DIR / "chinese_nouns.csv", index=False, encoding="utf-8")
#     zh_freq.to_csv(DATA_DIR / "chinese_noun_freq.csv", index=False, encoding="utf-8")
#     print(f"  [ZH] {zh_filtered['lemma'].nunique():,} lemmas ≥ {MIN_FREQ} occurrences retained.")

#     # ── Summary ───────────────────────────────────────────────────
#     print("\n── Preprocessing Summary ──")
#     print(f"  EN noun tokens (filtered) : {len(en_filtered):,}")
#     print(f"  EN unique lemmas          : {en_filtered['lemma'].nunique():,}")
#     print(f"  ZH noun tokens (filtered) : {len(zh_filtered):,}")
#     print(f"  ZH unique lemmas          : {zh_filtered['lemma'].nunique():,}")

#     print("\n  Top 10 English nouns:")
#     print(en_freq.head(10).to_string(index=False))
#     print("\n  Top 10 Chinese nouns:")
#     print(zh_freq.head(10).to_string(index=False))

#     print("\n✓ Step 2 complete.\n")


# if __name__ == "__main__":
#     main()


In [6]:
print("=" * 60)
print("Step 2: Preprocessing")
print("=" * 60)

# ── English ──────────────────────────────────────────────────
en_raw = preprocess_english(DATA_DIR / "english_verses.csv")
en_filtered, en_freq = apply_frequency_filter(en_raw)
en_filtered.to_csv(DATA_DIR / "english_nouns.csv", index=False, encoding="utf-8")
en_freq.to_csv(DATA_DIR / "english_noun_freq.csv", index=False, encoding="utf-8")
print(f"  [EN] {en_filtered['lemma'].nunique():,} lemmas ≥ {MIN_FREQ} occurrences retained.")

# ── Chinese ───────────────────────────────────────────────────
zh_raw = preprocess_chinese(DATA_DIR / "chinese_verses.csv")
zh_filtered, zh_freq = apply_frequency_filter(zh_raw)
zh_filtered.to_csv(DATA_DIR / "chinese_nouns.csv", index=False, encoding="utf-8")
zh_freq.to_csv(DATA_DIR / "chinese_noun_freq.csv", index=False, encoding="utf-8")
print(f"  [ZH] {zh_filtered['lemma'].nunique():,} lemmas ≥ {MIN_FREQ} occurrences retained.")

# ── Summary ───────────────────────────────────────────────────
print("\n── Preprocessing Summary ──")
print(f"  EN noun tokens (filtered) : {len(en_filtered):,}")
print(f"  EN unique lemmas          : {en_filtered['lemma'].nunique():,}")
print(f"  ZH noun tokens (filtered) : {len(zh_filtered):,}")
print(f"  ZH unique lemmas          : {zh_filtered['lemma'].nunique():,}")

print("\n  Top 10 English nouns:")
print(en_freq.head(10).to_string(index=False))
print("\n  Top 10 Chinese nouns:")
print(zh_freq.head(10).to_string(index=False))

print("\n✓ Step 2 complete.\n")


Step 2: Preprocessing
  [EN] Loading spaCy model…
  [EN] Processing 31,088 verses…
    … 31,088/31,088 verses  (538 v/s)  ETA 0.0 min  nouns so far: 116,878
  [EN] Done in 1.0 min. Extracted 116,878 noun occurrences.
  [EN] 636 lemmas ≥ 30 occurrences retained.
  [ZH] Loaded biblical proper name dictionary: jieba_biblical_dict.txt
  [ZH] Processing 31,069 verses with jieba…
    … 31,069/31,069 verses  (38 v/s)  ETA 0.0 min  nouns so far: 101,261
  [ZH] Done in 13.7 min. Extracted 101,261 noun occurrences.
  [ZH] 572 lemmas ≥ 30 occurrences retained.

── Preprocessing Summary ──
  EN noun tokens (filtered) : 98,195
  EN unique lemmas          : 636
  ZH noun tokens (filtered) : 72,998
  ZH unique lemmas          : 572

  Top 10 English nouns:
   lemma  count
     man   3927
     son   2929
    king   2533
  people   2376
     day   2037
    land   1501
    hand   1296
  father   1289
   house   1054
offering   1026

  Top 10 Chinese nouns:
lemma  count
    人   9184
   兒子   2356
    事   

# Step 3: Extract Context Embeddings

In [ ]:
!pip install torch transformers numpy pandas --break-system-packages

In [8]:
import os
import numpy as np
import pandas as pd
import torch
from pathlib import Path
from transformers import AutoTokenizer, AutoModel
from typing import List, Tuple

# ─── Configuration ────────────────────────────────────────────────────────────

DATA_DIR    = Path("/content") / "bible_data"
MODEL_NAME  = "xlm-roberta-base"   # Multilingual; same model for EN and ZH
BATCH_SIZE  = 32                   # Reduce to 8-16 if OOM on CPU
LAYERS      = [-1, -2, -3, -4]     # Last 4 layers averaged (standard WSI practice)
DEVICE      = "cuda" if torch.cuda.is_available() else "cpu"
MAX_SEQ_LEN = 128                  # Max subword tokens per sentence

print(f"Using device: {DEVICE}")

# ─── Model Loading ────────────────────────────────────────────────────────────

def load_model():
    print(f"  [model] Loading {MODEL_NAME}…")
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    model     = AutoModel.from_pretrained(MODEL_NAME, output_hidden_states=True)
    model.to(DEVICE)
    model.eval()
    return tokenizer, model


# ─── Embedding Extraction ─────────────────────────────────────────────────────

def get_target_embedding(
    tokenizer,
    model,
    sentences:  List[str],
    target_words: List[str],
) -> np.ndarray:
    """
    For each (sentence, target_word) pair, extract the contextual embedding
    of the target by:
      1. Tokenizing the sentence
      2. Finding subword token positions for the target word
      3. Averaging hidden states across the last 4 layers at those positions
      4. Mean-pooling across subwords for multi-token targets

    Returns: np.ndarray of shape (N, hidden_dim)
    """
    all_embeddings = []

    for i in range(0, len(sentences), BATCH_SIZE):
        batch_sents  = sentences[i : i + BATCH_SIZE]
        batch_targets = target_words[i : i + BATCH_SIZE]

        encoded = tokenizer(
            batch_sents,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=MAX_SEQ_LEN,
            return_offsets_mapping=True,
        )
        offset_mappings = encoded.pop("offset_mapping")  # not passed to model

        encoded = {k: v.to(DEVICE) for k, v in encoded.items()}

        with torch.no_grad():
            outputs = model(**encoded)

        # Stack selected hidden layers: shape (n_layers, batch, seq_len, hidden)
        hidden_states = torch.stack(
            [outputs.hidden_states[l] for l in LAYERS], dim=0
        )
        # Mean over selected layers: (batch, seq_len, hidden)
        layer_mean = hidden_states.mean(dim=0).cpu().numpy()

        input_ids = encoded["input_ids"].cpu().numpy()

        for j, (target, offsets_j) in enumerate(zip(batch_targets, offset_mappings)):
            # Re-encode the target word alone to find its subword tokens
            target_enc = tokenizer.encode(
                target, add_special_tokens=False
            )
            # Find target subword positions in the sentence encoding
            target_positions = _find_subword_positions(
                input_ids[j].tolist(), target_enc
            )
            if target_positions:
                token_emb = layer_mean[j][target_positions].mean(axis=0)
            else:
                # Fallback: mean-pool entire sequence (excluding [CLS]/[SEP])
                seq_len = (input_ids[j] != tokenizer.pad_token_id).sum()
                token_emb = layer_mean[j][1 : seq_len - 1].mean(axis=0)

            all_embeddings.append(token_emb)

        if (i // BATCH_SIZE) % 10 == 0:
            print(f"    … batch {i//BATCH_SIZE} / {len(sentences)//BATCH_SIZE}", end="\r")

    embeddings = np.array(all_embeddings, dtype=np.float32)
    # L2 normalize for cosine-based clustering
    norms = np.linalg.norm(embeddings, axis=1, keepdims=True)
    norms = np.where(norms == 0, 1, norms)
    return embeddings / norms


def _find_subword_positions(
    sentence_ids: List[int], target_ids: List[int]
) -> List[int]:
    """Find the start position of `target_ids` as a subsequence in `sentence_ids`."""
    n, m = len(sentence_ids), len(target_ids)
    for start in range(n - m + 1):
        if sentence_ids[start : start + m] == target_ids:
            return list(range(start, start + m))
    return []


# ─── Per-language Pipeline ────────────────────────────────────────────────────

def extract_embeddings_for_language(
    lang: str,
    noun_csv: Path,
    tokenizer,
    model,
) -> None:
    """Load nouns, extract embeddings, and save as .npz."""
    out_path = DATA_DIR / f"{lang}_embeddings.npz"
    if out_path.exists():
        print(f"  [{lang.upper()}] Embeddings already exist — skipping.")
        return

    df = pd.read_csv(noun_csv)
    print(f"  [{lang.upper()}] Extracting embeddings for {len(df):,} noun occurrences…")

    embeddings = get_target_embedding(
        tokenizer,
        model,
        sentences    = df["context"].tolist(),
        target_words = df["token"].tolist(),
    )
    print()  # newline after progress indicator

    np.savez_compressed(
        out_path,
        embeddings = embeddings,
        lemmas     = df["lemma"].to_numpy(dtype=str),
        verse_ids  = df["verse_id"].to_numpy(dtype=str),
        tokens     = df["token"].to_numpy(dtype=str),
    )
    print(f"  [{lang.upper()}] Saved {embeddings.shape} embeddings → {out_path.name}")


# ─── Main ─────────────────────────────────────────────────────────────────────

# def main():
#     print("=" * 60)
#     print("Step 3: Contextual Embedding Extraction (XLM-R)")
#     print("=" * 60)

#     tokenizer, model = load_model()

#     extract_embeddings_for_language(
#         "english",
#         DATA_DIR / "english_nouns.csv",
#         tokenizer, model,
#     )
#     extract_embeddings_for_language(
#         "chinese",
#         DATA_DIR / "chinese_nouns.csv",
#         tokenizer, model,
#     )

#     print("\n✓ Step 3 complete.\n")


# if __name__ == "__main__":
#     main()


Using device: cuda


In [9]:
print("=" * 60)
print("Step 3: Contextual Embedding Extraction (XLM-R)")
print("=" * 60)

tokenizer, model = load_model()

extract_embeddings_for_language(
    "english",
    DATA_DIR / "english_nouns.csv",
    tokenizer, model,
)
extract_embeddings_for_language(
    "chinese",
    DATA_DIR / "chinese_nouns.csv",
    tokenizer, model,
)

print("\n✓ Step 3 complete.\n")


Step 3: Contextual Embedding Extraction (XLM-R)
  [model] Loading xlm-roberta-base…


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Loading weights:   0%|          | 0/199 [00:00<?, ?it/s]

XLMRobertaModel LOAD REPORT from: xlm-roberta-base
Key                       | Status     |  | 
--------------------------+------------+--+-
lm_head.dense.bias        | UNEXPECTED |  | 
lm_head.dense.weight      | UNEXPECTED |  | 
lm_head.layer_norm.bias   | UNEXPECTED |  | 
lm_head.layer_norm.weight | UNEXPECTED |  | 
lm_head.bias              | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


  [ENGLISH] Extracting embeddings for 98,195 noun occurrences…

  [ENGLISH] Saved (98195, 768) embeddings → english_embeddings.npz
  [CHINESE] Extracting embeddings for 72,998 noun occurrences…

  [CHINESE] Saved (72998, 768) embeddings → chinese_embeddings.npz

✓ Step 3 complete.



# Step 4: WSI Clustering

In [ ]:
!pip install scikit-learn numpy pandas --break-system-packages

In [11]:
"""
Step 4: Word Sense Induction (WSI) via Agglomerative Clustering
================================================================
For each lemma in each language, clusters its contextual embeddings
to induce word senses. The number of clusters k is determined
automatically using the Silhouette Score (range 2–8) or set to 1
if the word shows insufficient sense variation.

Two clustering algorithms are run for robustness comparison:
  (A) Agglomerative Hierarchical Clustering (Ward linkage)    — primary
  (B) K-Means with k++ initialization                         — secondary

Outputs:
  - data/english_wsi_results.csv  : lemma, k_ward, k_kmeans, silhouette_ward, ...
  - data/chinese_wsi_results.csv  : same
  - data/english_sense_labels.csv : lemma, verse_id, cluster_ward, cluster_kmeans
  - data/chinese_sense_labels.csv : same

Usage:
  pip install scikit-learn numpy pandas --break-system-packages
  python 04_wsi_clustering.py

Design decisions (paper §3.3):
  - Ward linkage is preferred for lexical WSI (Ustalov et al. 2019)
  - k range: 1–8 senses; beyond 8 is linguistically implausible for
    the narrow domain of biblical text
  - Silhouette threshold: if best_silhouette < SILHOUETTE_THRESHOLD,
    k is set to 1 (monosemous)
  - UMAP dimensionality reduction to 50D before clustering improves
    silhouette stability (McInnes et al. 2018)
"""

import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.cluster import AgglomerativeClustering, KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import normalize
from typing import Tuple, Dict
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

# ─── Configuration ────────────────────────────────────────────────────────────

DATA_DIR    = Path("/content") / "bible_data"
# DATA_DIR             = Path(__file__).parent.parent / "data"
K_RANGE              = range(2, 9)           # Test k = 2, 3, …, 8
SILHOUETTE_THRESHOLD = 0.05                  # Below this → monosemous (k=1)
MIN_INSTANCES        = 5                     # Min occurrences to attempt clustering
USE_UMAP             = True                  # Reduce to 50D before clustering
UMAP_N_COMPONENTS    = 50
RANDOM_STATE         = 42


# ─── Optional UMAP reduction ─────────────────────────────────────────────────

def reduce_embeddings(embeddings: np.ndarray) -> np.ndarray:
    """
    Optionally reduce embedding dimensionality with UMAP before clustering.
    UMAP (McInnes et al. 2018) improves cluster separation for high-dim data.
    Falls back to PCA if umap-learn is not installed or if UMAP fails.
    """
    if not USE_UMAP or embeddings.shape[0] <= MIN_INSTANCES: # Added <= MIN_INSTANCES for robustness
        return embeddings

    try:
        import umap
        # Ensure n_components is always less than the number of samples
        n_components_umap = min(UMAP_N_COMPONENTS, embeddings.shape[0] - 1)
        if n_components_umap <= 0: # Handle cases where n_samples is 1
            return embeddings

        reducer = umap.UMAP(
            n_components=n_components_umap,
            metric="cosine",
            random_state=RANDOM_STATE,
            n_jobs=1,
        )
        return reducer.fit_transform(embeddings)
    except (ImportError, TypeError) as e: # Catch both ImportError and TypeError
        if isinstance(e, ImportError):
            print("    [warn] umap-learn not found — using PCA fallback. "
                  "Install: pip install umap-learn --break-system-packages")
        else:
            print(f"    [warn] UMAP failed ({e}) — using PCA fallback for this lemma. "
                  f"Embeddings shape: {embeddings.shape}")
        from sklearn.decomposition import PCA
        # Ensure n_components for PCA is also less than number of samples and features
        n_components_pca = min(UMAP_N_COMPONENTS, embeddings.shape[0] - 1, embeddings.shape[1])
        if n_components_pca <= 0: # Handle cases where n_samples is 1
            return embeddings
        return PCA(n_components=n_components_pca, random_state=RANDOM_STATE).fit_transform(embeddings)


# ─── Core WSI for a single lemma ─────────────────────────────────────────────

def induce_senses_for_lemma(
    embeddings: np.ndarray,
) -> Tuple[int, int, float, np.ndarray, np.ndarray]:
    """
    Given embeddings for all occurrences of a lemma, find the optimal k
    using silhouette score for both Ward and KMeans clustering.

    Returns:
        k_ward, k_kmeans, best_silhouette_ward,
        labels_ward (np.ndarray), labels_kmeans (np.ndarray)
    """
    n = len(embeddings)

    # Insufficient data → monosemous
    if n < MIN_INSTANCES:
        ones = np.zeros(n, dtype=int)
        return 1, 1, 0.0, ones, ones

    reduced = reduce_embeddings(embeddings)

    best_k_ward, best_sil_ward, best_labels_ward   = 1, -1.0, np.zeros(n, dtype=int)
    best_k_km,   best_sil_km,   best_labels_km     = 1, -1.0, np.zeros(n, dtype=int)

    for k in K_RANGE:
        if k >= n:
            break  # Can't have more clusters than data points

        # Ward agglomerative
        try:
            ward = AgglomerativeClustering(n_clusters=k, linkage="ward")
            labels_w = ward.fit_predict(reduced)
            if len(np.unique(labels_w)) > 1:
                sil_w = silhouette_score(reduced, labels_w, metric="euclidean",
                                         sample_size=min(1000, n))
                if sil_w > best_sil_ward:
                    best_sil_ward, best_k_ward, best_labels_ward = sil_w, k, labels_w
        except Exception:
            pass

        # K-Means
        try:
            km = KMeans(n_clusters=k, random_state=RANDOM_STATE,
                        n_init=10, max_iter=300)
            labels_k = km.fit_predict(reduced)
            if len(np.unique(labels_k)) > 1:
                sil_k = silhouette_score(reduced, labels_k, metric="euclidean",
                                          sample_size=min(1000, n))
                if sil_k > best_sil_km:
                    best_sil_km, best_k_km, best_labels_km = sil_k, k, labels_k
        except Exception:
            pass

    # Apply monosemy threshold
    if best_sil_ward < SILHOUETTE_THRESHOLD:
        best_k_ward    = 1
        best_labels_ward = np.zeros(n, dtype=int)

    if best_sil_km < SILHOUETTE_THRESHOLD:
        best_k_km    = 1
        best_labels_km = np.zeros(n, dtype=int)

    return (best_k_ward, best_k_km, max(best_sil_ward, 0.0),
            best_labels_ward, best_labels_km)


# ─── Per-language WSI Pipeline ───────────────────────────────────────────────

def run_wsi_for_language(lang: str) -> None:
    """Load embeddings, run WSI for every lemma, save results."""
    emb_path     = DATA_DIR / f"{lang}_embeddings.npz"
    result_path  = DATA_DIR / f"{lang}_wsi_results.csv"
    labels_path  = DATA_DIR / f"{lang}_sense_labels.csv"

    if result_path.exists():
        print(f"  [{lang.upper()}] WSI results already exist — skipping.")
        return

    print(f"  [{lang.upper()}] Loading embeddings…")
    data      = np.load(emb_path, allow_pickle=True)
    embs      = data["embeddings"].astype(np.float32)
    lemmas    = data["lemmas"].astype(str)
    verse_ids = data["verse_ids"].astype(str)
    tokens    = data["tokens"].astype(str)

    unique_lemmas = np.unique(lemmas)
    print(f"  [{lang.upper()}] Running WSI for {len(unique_lemmas):,} lemmas…")

    summary_rows = []
    label_rows   = []

    for i, lemma in enumerate(unique_lemmas):
        mask      = lemmas == lemma
        lemma_emb = embs[mask]
        n         = len(lemma_emb)

        k_ward, k_km, sil_ward, lbl_ward, lbl_km = induce_senses_for_lemma(lemma_emb)

        summary_rows.append({
            "lemma":            lemma,
            "n_occurrences":    n,
            "k_ward":           k_ward,
            "k_kmeans":         k_km,
            "silhouette_ward":  round(sil_ward, 4),
            "polysemous_ward":  int(k_ward > 1),
            "polysemous_kmeans":int(k_km   > 1),
        })

        # Save per-occurrence sense labels
        vid_subset = verse_ids[mask]
        tok_subset = tokens[mask]
        for j in range(n):
            label_rows.append({
                "lemma":        lemma,
                "verse_id":     vid_subset[j],
                "token":        tok_subset[j],
                "cluster_ward": lbl_ward[j],
                "cluster_kmeans": lbl_km[j],
            })

        if i % 50 == 0:
            print(f"    … {i:,}/{len(unique_lemmas):,} lemmas processed", end="\r")

    print()

    results_df = pd.DataFrame(summary_rows)
    labels_df  = pd.DataFrame(label_rows)

    results_df.to_csv(result_path, index=False, encoding="utf-8")
    labels_df.to_csv(labels_path,  index=False, encoding="utf-8")

    n_poly = (results_df["k_ward"] > 1).sum()
    mean_k = results_df["k_ward"].mean()
    print(f"  [{lang.upper()}] Done. Mean k_ward={mean_k:.3f} | Polysemous lemmas: {n_poly}/{len(unique_lemmas)}")


# ─── Main ─────────────────────────────────────────────────────────────────────

# def main():
#     print("=" * 60)
#     print("Step 4: Word Sense Induction (WSI)")
#     print("=" * 60)

#     run_wsi_for_language("english")
#     run_wsi_for_language("chinese")

#     # Quick comparison preview
#     en = pd.read_csv(DATA_DIR / "english_wsi_results.csv")
#     zh = pd.read_csv(DATA_DIR / "chinese_wsi_results.csv")

#     print("\n── Quick Comparison Preview ──")
#     print(f"  EN | mean senses/lemma (Ward) : {en['k_ward'].mean():.3f}")
#     print(f"  ZH | mean senses/lemma (Ward) : {zh['k_ward'].mean():.3f}")
#     print(f"  EN | % polysemous lemmas       : {(en['k_ward'] > 1).mean()*100:.1f}%")
#     print(f"  ZH | % polysemous lemmas       : {(zh['k_ward'] > 1).mean()*100:.1f}%")

#     print("\n✓ Step 4 complete.\n")


# if __name__ == "__main__":
#     main()


In [ ]:
print("=" * 60)
print("Step 4: Word Sense Induction (WSI)")
print("=" * 60)

run_wsi_for_language("english")
run_wsi_for_language("chinese")

# Quick comparison preview
en = pd.read_csv(DATA_DIR / "english_wsi_results.csv")
zh = pd.read_csv(DATA_DIR / "chinese_wsi_results.csv")

print("\n── Quick Comparison Preview ──")
print(f"  EN | mean senses/lemma (Ward) : {en['k_ward'].mean():.3f}")
print(f"  ZH | mean senses/lemma (Ward) : {zh['k_ward'].mean():.3f}")
print(f"  EN | % polysemous lemmas       : {(en['k_ward'] > 1).mean()*100:.1f}%")
print(f"  ZH | % polysemous lemmas       : {(zh['k_ward'] > 1).mean()*100:.1f}%")

print("\n✓ Step 4 complete.\n")

# Step 5: Validation and Statistical Analysis

In [ ]:
!pip install nltk scipy matplotlib seaborn pandas numpy --break-system-packages

In [14]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [15]:
"""
Step 5: Validation and Statistical Analysis
============================================
Validates WSI-induced sense counts against gold-standard lexical resources:
  - English: Princeton WordNet (via NLTK)
  - Chinese:  Chinese WordNet (via Taiwanese CWN or HowNet)

Also performs the core statistical comparison between languages:
  - Mann-Whitney U test (non-parametric, appropriate for skewed count data)
  - Cohen's d effect size
  - Spearman correlation with WordNet sense counts (validation)
  - Distribution plots (saved as PNG for inclusion in paper)

Outputs:
  - output/validation_correlation_en.csv
  - output/validation_correlation_zh.csv   (if CWN available)
  - output/statistical_comparison.csv
  - output/figures/sense_distribution_en.png
  - output/figures/sense_distribution_zh.png
  - output/figures/comparison_boxplot.png
  - output/figures/wordnet_correlation_en.png

Usage:
  pip install nltk scipy matplotlib seaborn pandas numpy --break-system-packages
  python -c "import nltk; nltk.download('wordnet')"
  python 05_validation_statistics.py
"""

import numpy as np
import pandas as pd
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from scipy import stats
from typing import Optional, Tuple

# ─── Configuration ────────────────────────────────────────────────────────────

DATA_DIR    = Path("/content") / "bible_data"
OUTPUT_DIR = Path("/content") / "output"
# DATA_DIR   = Path(__file__).parent.parent / "data"
# OUTPUT_DIR = Path(__file__).parent.parent / "output"
FIG_DIR    = OUTPUT_DIR / "figures"
OUTPUT_DIR.mkdir(exist_ok=True)
FIG_DIR.mkdir(exist_ok=True)

plt.rcParams.update({
    "font.family":  "DejaVu Sans",
    "font.size":    11,
    "axes.titlesize": 13,
    "axes.labelsize": 12,
})


# ─── WordNet Validation (English) ────────────────────────────────────────────

def get_wordnet_sense_counts(lemmas: list) -> dict:
    """
    Look up the number of synsets for each lemma in Princeton WordNet.
    Noun synsets only (pos='n').
    """
    try:
        from nltk.corpus import wordnet as wn
    except ImportError:
        raise ImportError("Run: pip install nltk --break-system-packages && "
                          "python -c \"import nltk; nltk.download('wordnet')\"")

    counts = {}
    for lemma in lemmas:
        synsets = wn.synsets(lemma.lower(), pos=wn.NOUN)
        counts[lemma] = len(synsets)
    return counts


def validate_english(en_results: pd.DataFrame) -> pd.DataFrame:
    """
    Correlate WSI-induced k_ward with WordNet sense counts for English nouns.
    Returns a merged DataFrame with both counts.
    """
    print("  [validate] Looking up Princeton WordNet sense counts…")
    lemmas = en_results["lemma"].tolist()
    wn_counts = get_wordnet_sense_counts(lemmas)

    en_results = en_results.copy()
    en_results["wn_senses"] = en_results["lemma"].map(wn_counts).fillna(0).astype(int)

    # Keep only lemmas with at least 1 WordNet entry
    valid = en_results[en_results["wn_senses"] > 0].copy()

    rho, p = stats.spearmanr(valid["k_ward"], valid["wn_senses"])
    print(f"  [validate] EN Spearman ρ(k_ward, WN_senses) = {rho:.3f}  p = {p:.4f}  "
          f"(n={len(valid)})")

    return valid, rho, p


# ─── Statistical Comparison ──────────────────────────────────────────────────

def mann_whitney_comparison(
    en_k: np.ndarray,
    zh_k: np.ndarray,
) -> dict:
    """
    Mann-Whitney U test comparing mean sense counts between EN and ZH.
    Also computes Cohen's d and the common language effect size (CLES).
    """
    u_stat, p_val = stats.mannwhitneyu(en_k, zh_k, alternative="two-sided")
    n1, n2        = len(en_k), len(zh_k)
    cles          = u_stat / (n1 * n2)  # Common Language Effect Size

    # Cohen's d (for reference alongside CLES)
    pooled_std = np.sqrt(
        ((n1 - 1) * en_k.std(ddof=1) ** 2 + (n2 - 1) * zh_k.std(ddof=1) ** 2)
        / (n1 + n2 - 2)
    )
    cohens_d = (en_k.mean() - zh_k.mean()) / (pooled_std + 1e-9)

    return {
        "en_mean_k":   round(en_k.mean(), 4),
        "zh_mean_k":   round(zh_k.mean(), 4),
        "en_median_k": round(float(np.median(en_k)), 4),
        "zh_median_k": round(float(np.median(zh_k)), 4),
        "en_std_k":    round(en_k.std(ddof=1), 4),
        "zh_std_k":    round(zh_k.std(ddof=1), 4),
        "U_statistic": round(u_stat, 2),
        "p_value":     round(p_val, 6),
        "CLES":        round(cles, 4),
        "cohens_d":    round(cohens_d, 4),
        "n_en":        int(n1),
        "n_zh":        int(n2),
    }


# ─── Figures ─────────────────────────────────────────────────────────────────

def plot_sense_distribution(df: pd.DataFrame, lang: str, col: str = "k_ward") -> None:
    fig, ax = plt.subplots(figsize=(8, 4))
    counts = df[col].value_counts().sort_index()
    ax.bar(counts.index, counts.values, color="#4C72B0", edgecolor="white", linewidth=0.5)
    ax.set_xlabel("Number of Induced Senses (k)")
    ax.set_ylabel("Number of Lemmas")
    ax.set_title(f"{lang.capitalize()} — Distribution of Induced Senses per Noun Lemma")
    ax.set_xticks(range(1, df[col].max() + 1))
    fig.tight_layout()
    path = FIG_DIR / f"sense_distribution_{lang}.png"
    fig.savefig(path, dpi=150)
    plt.close(fig)
    print(f"  [fig] Saved: {path.name}")


def plot_comparison_boxplot(en_df: pd.DataFrame, zh_df: pd.DataFrame) -> None:
    combined = pd.concat([
        en_df[["k_ward"]].assign(Language="English"),
        zh_df[["k_ward"]].assign(Language="Chinese"),
    ])
    fig, ax = plt.subplots(figsize=(7, 5))
    sns.violinplot(data=combined, x="Language", y="k_ward",
                   palette=["#4C72B0", "#DD8452"], inner="box", ax=ax)
    ax.set_ylabel("Induced Senses per Lemma (k, Ward)")
    ax.set_title("Distribution of Polysemy Degree: English vs. Chinese Common Nouns\n"
                 "(Bible Parallel Corpus, WSI via XLM-R + Agglomerative Clustering)")
    fig.tight_layout()
    path = FIG_DIR / "comparison_violinplot.png"
    fig.savefig(path, dpi=150)
    plt.close(fig)
    print(f"  [fig] Saved: {path.name}")


def plot_wordnet_correlation(valid_df: pd.DataFrame, rho: float, p: float) -> None:
    fig, ax = plt.subplots(figsize=(6, 5))
    ax.scatter(valid_df["wn_senses"], valid_df["k_ward"],
               alpha=0.4, s=20, color="#4C72B0")
    ax.set_xlabel("WordNet Noun Synset Count")
    ax.set_ylabel("WSI-Induced k (Ward)")
    ax.set_title(f"Validation: WSI k vs. WordNet Senses (English Nouns)\n"
                 f"Spearman ρ = {rho:.3f}, p = {p:.4f}")
    # Trend line
    m, b = np.polyfit(valid_df["wn_senses"], valid_df["k_ward"], 1)
    x_line = np.linspace(valid_df["wn_senses"].min(), valid_df["wn_senses"].max(), 100)
    ax.plot(x_line, m * x_line + b, color="red", linewidth=1.5, linestyle="--")
    fig.tight_layout()
    path = FIG_DIR / "wordnet_correlation_en.png"
    fig.savefig(path, dpi=150)
    plt.close(fig)
    print(f"  [fig] Saved: {path.name}")


# ─── Main ─────────────────────────────────────────────────────────────────────

# def main():
#     print("=" * 60)
#     print("Step 5: Validation and Statistical Analysis")
#     print("=" * 60)

#     en = pd.read_csv(DATA_DIR / "english_wsi_results.csv")
#     zh = pd.read_csv(DATA_DIR / "chinese_wsi_results.csv")

#     # ── Validation (English vs. WordNet) ──────────────────────────
#     valid_en, rho, p = validate_english(en)
#     valid_en.to_csv(OUTPUT_DIR / "validation_correlation_en.csv", index=False)

#     # ── Statistical Comparison ────────────────────────────────────
#     stats_result = mann_whitney_comparison(
#         en["k_ward"].values,
#         zh["k_ward"].values,
#     )
#     stats_df = pd.DataFrame([stats_result])
#     stats_df.to_csv(OUTPUT_DIR / "statistical_comparison.csv", index=False)

#     print("\n── Statistical Comparison Results ──")
#     for k, v in stats_result.items():
#         print(f"  {k:22s}: {v}")

#     # ── Figures ───────────────────────────────────────────────────
#     plot_sense_distribution(en, "english")
#     plot_sense_distribution(zh, "chinese")
#     plot_comparison_boxplot(en, zh)
#     plot_wordnet_correlation(valid_en, rho, p)

#     # ── Interpretation ────────────────────────────────────────────
#     print("\n── Interpretation ──")
#     if stats_result["p_value"] < 0.05:
#         direction = "English" if stats_result["en_mean_k"] > stats_result["zh_mean_k"] else "Chinese"
#         print(f"  Significant difference found (p={stats_result['p_value']:.4f}).")
#         print(f"  {direction} nouns show higher mean polysemy degree.")
#     else:
#         print(f"  No significant difference found (p={stats_result['p_value']:.4f}).")

#     d = abs(stats_result["cohens_d"])
#     magnitude = "small" if d < 0.2 else ("medium" if d < 0.5 else "large")
#     print(f"  Effect size: Cohen's d = {stats_result['cohens_d']:.3f} ({magnitude})")
#     print(f"  Spearman ρ (EN WSI vs. WordNet): {rho:.3f} (p={p:.4f})")

#     print("\n✓ Step 5 complete.\n")


# if __name__ == "__main__":
#     main()


In [16]:
print("=" * 60)
print("Step 5: Validation and Statistical Analysis")
print("=" * 60)

en = pd.read_csv(DATA_DIR / "english_wsi_results.csv")
zh = pd.read_csv(DATA_DIR / "chinese_wsi_results.csv")

# ── Validation (English vs. WordNet) ──────────────────────────
valid_en, rho, p = validate_english(en)
valid_en.to_csv(OUTPUT_DIR / "validation_correlation_en.csv", index=False)

# ── Statistical Comparison ────────────────────────────────────
stats_result = mann_whitney_comparison(
    en["k_ward"].values,
    zh["k_ward"].values,
)
stats_df = pd.DataFrame([stats_result])
stats_df.to_csv(OUTPUT_DIR / "statistical_comparison.csv", index=False)

print("\n── Statistical Comparison Results ──")
for k, v in stats_result.items():
    print(f"  {k:22s}: {v}")

# ── Figures ───────────────────────────────────────────────────
plot_sense_distribution(en, "english")
plot_sense_distribution(zh, "chinese")
plot_comparison_boxplot(en, zh)
plot_wordnet_correlation(valid_en, rho, p)

# ── Interpretation ────────────────────────────────────────────
print("\n── Interpretation ──")
if stats_result["p_value"] < 0.05:
    direction = "English" if stats_result["en_mean_k"] > stats_result["zh_mean_k"] else "Chinese"
    print(f"  Significant difference found (p={stats_result['p_value']:.4f}).")
    print(f"  {direction} nouns show higher mean polysemy degree.")
else:
    print(f"  No significant difference found (p={stats_result['p_value']:.4f}).")

d = abs(stats_result["cohens_d"])
magnitude = "small" if d < 0.2 else ("medium" if d < 0.5 else "large")
print(f"  Effect size: Cohen's d = {stats_result['cohens_d']:.3f} ({magnitude})")
print(f"  Spearman ρ (EN WSI vs. WordNet): {rho:.3f} (p={p:.4f})")

print("\n✓ Step 5 complete.\n")


Step 5: Validation and Statistical Analysis
  [validate] Looking up Princeton WordNet sense counts…
  [validate] EN Spearman ρ(k_ward, WN_senses) = 0.020  p = 0.6191  (n=630)

── Statistical Comparison Results ──
  en_mean_k             : 3.3459
  zh_mean_k             : 3.4738
  en_median_k           : 2.0
  zh_median_k           : 2.0
  en_std_k              : 2.0931
  zh_std_k              : 2.2647
  U_statistic           : 182813.5
  p_value               : 0.864495
  CLES                  : 0.5025
  cohens_d              : -0.0588
  n_en                  : 636
  n_zh                  : 572
  [fig] Saved: sense_distribution_english.png
  [fig] Saved: sense_distribution_chinese.png
  [fig] Saved: comparison_violinplot.png
  [fig] Saved: wordnet_correlation_en.png

── Interpretation ──
  No significant difference found (p=0.8645).
  Effect size: Cohen's d = -0.059 (small)
  Spearman ρ (EN WSI vs. WordNet): 0.020 (p=0.6191)

✓ Step 5 complete.



# Step 6: Qualitative Analysis

In [17]:
"""
Step 6: Qualitative Analysis — Sense Cluster Inspection
=========================================================
For each polysemous lemma, retrieves representative example verses
for each induced sense cluster. This supports the qualitative
analysis section of the paper, demonstrating that clusters correspond
to meaningful, interpretable senses.

Also generates a LaTeX-ready table of top polysemous words
for both languages (for paper Table 3).

Outputs:
  - output/qualitative_en_top_polysemous.txt   (sense examples)
  - output/qualitative_zh_top_polysemous.txt
  - output/table_top_polysemous_latex.tex       (LaTeX table)
  - output/polysemy_profile_comparison.csv      (wide comparison table)

Usage:
  python 06_qualitative_analysis.py
"""

import pandas as pd
import numpy as np
from pathlib import Path
from collections import defaultdict

# ─── Configuration ────────────────────────────────────────────────────────────

DATA_DIR    = Path("/content") / "bible_data"
OUTPUT_DIR = Path("/content") / "output"
# DATA_DIR   = Path(__file__).parent.parent / "data"
# OUTPUT_DIR = Path(__file__).parent.parent / "output"
OUTPUT_DIR.mkdir(exist_ok=True)

TOP_N_WORDS       = 20   # Top N most polysemous lemmas per language
EXAMPLES_PER_SENSE = 2   # Number of example verses per cluster


# ─── Sense Example Retrieval ─────────────────────────────────────────────────

def get_sense_examples(
    lang: str,
    top_n: int = TOP_N_WORDS,
    examples_per_sense: int = EXAMPLES_PER_SENSE,
) -> str:
    """
    For the top_n most polysemous lemmas, retrieve example contexts
    for each induced sense cluster.

    Returns a formatted string ready for a paper's qualitative appendix.
    """
    results_df = pd.read_csv(DATA_DIR / f"{lang}_wsi_results.csv")
    labels_df  = pd.read_csv(DATA_DIR / f"{lang}_sense_labels.csv")
    nouns_df   = pd.read_csv(DATA_DIR / f"{lang}_nouns.csv")

    # Top polysemous by k_ward
    poly = results_df[results_df["k_ward"] > 1].nlargest(top_n, "k_ward")

    # Merge labels with original contexts
    merged = labels_df.merge(
        nouns_df[["lemma", "verse_id", "context"]].drop_duplicates(),
        on=["lemma", "verse_id"],
        how="left",
    )

    lines = []
    lines.append(f"{'='*60}")
    lines.append(f"QUALITATIVE SENSE ANALYSIS — {lang.upper()}")
    lines.append(f"Top {top_n} Most Polysemous Nouns (WSI, Ward Clustering)")
    lines.append(f"{'='*60}\n")

    for _, row in poly.iterrows():
        lemma = row["lemma"]
        k     = int(row["k_ward"])
        n_occ = int(row["n_occurrences"])
        sil   = row.get("silhouette_ward", "N/A")

        lines.append(f"Lemma: '{lemma}'  |  k={k}  |  n={n_occ}  |  silhouette={sil}")
        lines.append("-" * 50)

        lemma_data = merged[merged["lemma"] == lemma]

        for cluster_id in range(k):
            cluster_rows = lemma_data[lemma_data["cluster_ward"] == cluster_id]
            lines.append(f"  Sense {cluster_id + 1} ({len(cluster_rows)} occurrences):")

            # Sample diverse examples
            sample = cluster_rows.dropna(subset=["context"]).head(examples_per_sense)
            for _, ex in sample.iterrows():
                ctx = str(ex["context"])[:200].replace("\n", " ")
                lines.append(f"    • {ex['verse_id']}: {ctx}")

        lines.append("")

    return "\n".join(lines)


# ─── LaTeX Table Generation ──────────────────────────────────────────────────

def generate_latex_table(top_n: int = 15) -> str:
    """
    Generate a LaTeX longtable comparing top polysemous nouns in both languages.
    Format:
      Rank | English Lemma | EN k | Chinese Lemma | ZH k
    """
    en = pd.read_csv(DATA_DIR / "english_wsi_results.csv")
    zh = pd.read_csv(DATA_DIR / "chinese_wsi_results.csv")

    en_top = en.nlargest(top_n, "k_ward")[["lemma", "k_ward", "n_occurrences"]].reset_index(drop=True)
    zh_top = zh.nlargest(top_n, "k_ward")[["lemma", "k_ward", "n_occurrences"]].reset_index(drop=True)

    lines = [
        r"\begin{table}[h]",
        r"\centering",
        r"\caption{Top Polysemous Common Nouns by Induced Sense Count (k): English vs. Chinese}",
        r"\label{tab:top_polysemous}",
        r"\begin{tabular}{clccclcc}",
        r"\toprule",
        r"Rank & English Lemma & EN $k$ & EN $n$ & & Chinese Lemma & ZH $k$ & ZH $n$ \\",
        r"\midrule",
    ]

    for i in range(top_n):
        en_row = en_top.iloc[i] if i < len(en_top) else None
        zh_row = zh_top.iloc[i] if i < len(zh_top) else None

        en_lemma = en_row["lemma"]                  if en_row is not None else ""
        en_k     = int(en_row["k_ward"])            if en_row is not None else ""
        en_n     = int(en_row["n_occurrences"])     if en_row is not None else ""
        zh_lemma = zh_row["lemma"]                  if zh_row is not None else ""
        zh_k     = int(zh_row["k_ward"])            if zh_row is not None else ""
        zh_n     = int(zh_row["n_occurrences"])     if zh_row is not None else ""

        lines.append(
            f"{i+1} & {en_lemma} & {en_k} & {en_n} & & {zh_lemma} & {zh_k} & {zh_n} \\\\"
        )

    lines += [
        r"\bottomrule",
        r"\end{tabular}",
        r"\end{table}",
    ]
    return "\n".join(lines)


# ─── Wide Comparison Profile ─────────────────────────────────────────────────

def generate_comparison_profile() -> pd.DataFrame:
    """
    Create a summary comparison table for paper Table 2.
    """
    en = pd.read_csv(DATA_DIR / "english_wsi_results.csv")
    zh = pd.read_csv(DATA_DIR / "chinese_wsi_results.csv")

    def profile(df: pd.DataFrame, lang: str) -> dict:
        return {
            "Language":            lang,
            "Total lemmas":        len(df),
            "Mean k (Ward)":       round(df["k_ward"].mean(), 3),
            "Median k (Ward)":     round(df["k_ward"].median(), 3),
            "Std k (Ward)":        round(df["k_ward"].std(ddof=1), 3),
            "% Monosemous (k=1)":  round((df["k_ward"] == 1).mean() * 100, 1),
            "% Polysemous (k>1)":  round((df["k_ward"] > 1).mean() * 100, 1),
            "Max k":               int(df["k_ward"].max()),
            "Mean k (KMeans)":     round(df["k_kmeans"].mean(), 3),
            "Agreement (Ward=KM)": round((df["k_ward"] == df["k_kmeans"]).mean() * 100, 1),
        }

    rows = [profile(en, "English"), profile(zh, "Chinese")]
    return pd.DataFrame(rows)


# ─── Main ─────────────────────────────────────────────────────────────────────

# def main():
#     print("=" * 60)
#     print("Step 6: Qualitative Analysis")
#     print("=" * 60)

#     # ── Sense examples ────────────────────────────────────────────
#     for lang in ["english", "chinese"]:
#         text = get_sense_examples(lang)
#         out  = OUTPUT_DIR / f"qualitative_{lang}_top_polysemous.txt"
#         out.write_text(text, encoding="utf-8")
#         print(f"  [saved] {out.name}")

#     # ── LaTeX table ───────────────────────────────────────────────
#     latex = generate_latex_table(top_n=15)
#     tex_path = OUTPUT_DIR / "table_top_polysemous_latex.tex"
#     tex_path.write_text(latex, encoding="utf-8")
#     print(f"  [saved] {tex_path.name}")

#     # ── Comparison profile ────────────────────────────────────────
#     profile = generate_comparison_profile()
#     csv_path = OUTPUT_DIR / "polysemy_profile_comparison.csv"
#     profile.to_csv(csv_path, index=False)
#     print(f"  [saved] {csv_path.name}")
#     print()
#     print(profile.to_string(index=False))

#     print("\n✓ Step 6 complete.\n")


# if __name__ == "__main__":
#     main()

In [18]:
print("=" * 60)
print("Step 6: Qualitative Analysis")
print("=" * 60)

# ── Sense examples ────────────────────────────────────────────
for lang in ["english", "chinese"]:
    text = get_sense_examples(lang)
    out  = OUTPUT_DIR / f"qualitative_{lang}_top_polysemous.txt"
    out.write_text(text, encoding="utf-8")
    print(f"  [saved] {out.name}")

# ── LaTeX table ───────────────────────────────────────────────
latex = generate_latex_table(top_n=15)
tex_path = OUTPUT_DIR / "table_top_polysemous_latex.tex"
tex_path.write_text(latex, encoding="utf-8")
print(f"  [saved] {tex_path.name}")

# ── Comparison profile ────────────────────────────────────────
profile = generate_comparison_profile()
csv_path = OUTPUT_DIR / "polysemy_profile_comparison.csv"
profile.to_csv(csv_path, index=False)
print(f"  [saved] {csv_path.name}")
print()
print(profile.to_string(index=False))

print("\n✓ Step 6 complete.\n")


Step 6: Qualitative Analysis
  [saved] qualitative_english_top_polysemous.txt
  [saved] qualitative_chinese_top_polysemous.txt
  [saved] table_top_polysemous_latex.tex
  [saved] polysemy_profile_comparison.csv

Language  Total lemmas  Mean k (Ward)  Median k (Ward)  Std k (Ward)  % Monosemous (k=1)  % Polysemous (k>1)  Max k  Mean k (KMeans)  Agreement (Ward=KM)
 English           636          3.346              2.0         2.093                 0.0               100.0      8            3.264                 81.4
 Chinese           572          3.474              2.0         2.265                 0.0               100.0      8            3.318                 78.7

✓ Step 6 complete.



In [ ]:
!zip -r output.zip /content/output/

In [ ]:
!zip -r nt_data_niv_cuv.zip /content/bible_data/